In [1]:
### ----- Imports ----- ###

# import sys
# sys.path.append('/home/sdybing/neic-mlaapde')

# from mlaapde.access import MLAAPDE_Access
# from mlaapde import UTC
import matplotlib.pyplot as plt
import numpy as np
import os.path
import os
import shutil
import glob
import h5py
import tensorflow as tf

# os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
# os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'


# mlpa = MLAAPDE_Access(data_dir = '/data/hank/mlaapde_subset/data', random_seed = 616) # 3 months
# dataset = 'subset'

# mlpa = MLAAPDE_Access(data_dir = '/data/hank/mlaapde_v1b/data', random_seed = 616)
dataset = 'v1b'

# mlpa.data_dir

In [2]:
from tensorflow.keras import layers
from tensorflow.keras import backend as K

# gpus = tf.config.list_physical_devices('GPU')
# if gpus:
#     try:
#         for gpu in gpus:
#             tf.config.experimental.set_memory_growth(gpu, True)
#         logical_gpus = tf.config.list_logical_devices('GPU')
#         print(len(gpus), "Physical GPU,", len(logical_gpus), "Logical GPUs")
#     except RuntimeError as e:
#         print(e)
#         raise

In [3]:
### ----- Parameters ----- ###

# Where to save the products
models_figs_path = '/home/sdybing/mlaapde/figures/'

# MLAAPDE/data generation params
#nsamp = False # Samples of waveforms to load from MLAAPDE
#n_train_samp = 1000000
#n_valid_samp = 200000
#nsamp = n_train_samp + n_valid_samp
sr = 40 # Sampling rate
trim_sec = 60 # Trimming amount around phase pick to get from MLAAPDE
trim_pre_sec = trim_sec
trim_post_sec = trim_pre_sec
window_len = trim_pre_sec + trim_post_sec
#train_split = 0.8 # Percentage of data used in training
#valid_split = 0.2 # Percentage of data used for validation
n_channels = 3 # Instrument channels
cut_lens = [7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 25, 30, 35, 40, 50, 60, 70, 80, 90, 100, 110, 120]
cut_lens_finish = [70, 80, 90, 100, 110, 120]
cut_lens_test = [7, 8]
desired_shift = 3
max_shift = desired_shift * 2 # Since the shifting method actually makes it half what this value is set to
min_snr_db = False
max_snr_db = False
log_progress_fraction = 100
valid_phases = ['P', 'Pn', 'Pg']
cast_dtype = np.float32

# Training/model params
epochs_number = 200
batch_size = int(256) # Reducing to help memory
monte_carlo_sampling = 50
drop_rate = 0.5
filters = [32, 64, 96, 128, 256] 

# Used if loading a trained model
training_dataset = 'v1b'
shift_status = 'shifted'
model_folder_path = '/home/sdybing/mlaapde/training/'

# To make end error plots
mean_errors = []
std_errors = []

In [4]:
hdf5_save_dir = '/hdd/mlaapde'

In [5]:
### ----- Load the full dataset from HDF5 files ----- ###

training_data = h5py.File(hdf5_save_dir + '/training_data.hdf5', 'r')
dataset_names = list(training_data.keys())
print(dataset_names)

train_waves = training_data['waves'][:]
train_mags = training_data['magnitude'][:]
train_phase_id = training_data['phase_id'][:]

validation_data = h5py.File(hdf5_save_dir + '/validation_data.hdf5', 'r')
valid_waves = validation_data['waves'][:]
valid_mags = validation_data['magnitude'][:]
valid_phase_id = training_data['phase_id'][:]

# testing_data = h5py.File(hdf5_save_dir + '/testing_data.hdf5', 'r')
# test_waves = testing_data['waves'][:]
# test_mags = testing_data['magnitude'][:]
# test_phase_id = testing_data['phase_id'][:]

training_data.close()
validation_data.close()
# testing_data.close()

['magnitude', 'magnitude_type', 'phase_id', 'snr_db', 'waves']


In [6]:
print(train_waves.shape)
print(train_mags.shape)
print(valid_waves.shape)
print(valid_mags.shape)
# print(test_waves.shape)
# print(test_mags.shape)

(2431341, 3, 4800)
(2431341,)
(489268, 3, 4800)
(489268,)


In [7]:
train_waves_t = train_waves.transpose(0,2,1)
valid_waves_t = valid_waves.transpose(0,2,1)
# test_waves_t = valid_waves.transpose(0,2,1)

print(train_waves_t.shape)
print(train_mags.shape)
print(valid_waves_t.shape)
print(valid_mags.shape)
# print(test_waves_t.shape)
# print(test_mags.shape)

(2431341, 4800, 3)
(2431341,)
(489268, 4800, 3)
(489268,)


In [8]:
del train_waves
del valid_waves
# del test_waves

In [9]:
# isfinite = np.isfinite(train_waves_t)
# #print(isfinite)
# i = np.where(isfinite == False)[0]
# print(i)

In [10]:
# k = np.unique(i)
# print(k)

In [11]:
idx = 741100
idx2 = 741102
# print(train_waves_t[idx])
# print(train_waves_t[idx2])
# plt.plot(train_waves_t[idx])
# plt.show()
# plt.plot(train_waves_t[idx2])
# plt.show();

In [12]:
# Fixing the weird nan wave

print(train_phase_id[idx])
print(train_waves_t[idx])
print(train_phase_id[idx2])
print(train_waves_t[idx2])

copy_wave = train_waves_t[0]
copy_mag = train_mags[0]
copy_wave2 = train_waves_t[1]
copy_mag2 = train_mags[1]

train_waves_t[idx] = copy_wave
train_mags[idx] = copy_mag
train_waves_t[idx2] = copy_wave2
train_mags[idx2] = copy_mag2

us20002njs_CI.BAR.BH*.--_P
[[nan nan nan]
 [nan nan nan]
 [nan nan nan]
 ...
 [nan nan nan]
 [nan nan nan]
 [nan nan nan]]
us20002njs_CI.MWC.BH*.--_P
[[nan nan nan]
 [nan nan nan]
 [nan nan nan]
 ...
 [nan nan nan]
 [nan nan nan]
 [nan nan nan]]


In [13]:
# Check to make sure it's good now

print(train_waves_t[idx])
print(train_mags[idx])
print(train_mags[0])

print(train_waves_t[idx2])
print(train_mags[idx2])
print(train_mags[1])

[[ 0.13537762 -0.07492092 -0.01111916]
 [ 0.14131626 -0.07153969 -0.02545009]
 [ 0.14370808 -0.06007923 -0.0382474 ]
 ...
 [ 0.06729667 -0.47734734 -0.44188842]
 [ 0.0809005  -0.47435188 -0.44014212]
 [ 0.07301358 -0.42699108 -0.44291207]]
4.1
4.1
[[-0.15383083 -0.07718702  0.07553101]
 [-0.16129185  0.03450812  0.13697691]
 [-0.1270593   0.10472912  0.11481622]
 ...
 [-0.18223436  0.15392898 -0.21175863]
 [-0.13242158  0.12540203 -0.03071801]
 [-0.04486514  0.09270573  0.1601974 ]]
4.1
4.1


In [22]:
n_train_samp = len(train_mags)
n_valid_samp = len(valid_mags)
nsamp = n_train_samp + n_valid_samp
print(nsamp)
data = 'validation'

2920609


In [ ]:
########## STUFF THAT NEEDS LOOPING ##########

debug = False

for cut_len in cut_lens:
    print('Cut len: ' + str(cut_len))
    
    ### ----- Where are the trained models/figures getting saved? ----- ###

    save_dir = models_figs_path + data + '/' + str(dataset) + '_' + str(nsamp) + 'samps_' + str(cut_len-6) + 's_window'
    if os.path.isdir(save_dir):
        pass
    else: # deletes directory to start over: shutil.rmtree(save_dir)  
        os.makedirs(save_dir)

    ### ----- Cut and shift validation data to match the training data ----- ###

    ## Cut ##
    if debug:
        rand = np.random.choice(np.arange(0,len(valid_mags),1))
        print('Rand: ' + str(rand))
        valid_times = np.arange(0, window_len, 1/sr)
        plt.figure(facecolor = 'white')
        plt.suptitle('Original validation data')
        plt.subplot(311)
        plt.plot(valid_times, valid_waves_t[rand,:,0], color = 'C0')
        plt.subplot(312)
        plt.plot(valid_times, valid_waves_t[rand,:,1], color = 'C1')
        plt.subplot(313)
        plt.plot(valid_times, valid_waves_t[rand,:,2], color = 'C2')
        plt.subplots_adjust(hspace = 0)
        plt.show();
    
    middle = int(valid_waves_t.shape[1] / 2)
    if debug: print('Middle: ' + str(middle))
    valid_size = int(n_valid_samp)
    if debug: print('Valid size: ' + str(valid_size))
    cut_valid_waves_t = np.zeros((valid_size, int(cut_len*sr), 3)) 
    if debug: print('Cut waves t shape: ' + str(cut_valid_waves_t.shape))

    for i in range(len(valid_waves_t)):
        cut_valid_waves_t[i,] = valid_waves_t[i, int(middle - (cut_len/2)*sr) : int(middle + (cut_len/2)*sr), 0:3]
    if debug: print('Cut waves t shape: ' + str(cut_valid_waves_t.shape))
    if debug:
        valid_cut_times = np.arange(0, cut_len, 1/sr)
        print('Rand: ' + str(rand))
        plt.figure(facecolor = 'white')
        plt.suptitle('Cut validation data')
        plt.subplot(311)
        plt.plot(valid_cut_times, cut_valid_waves_t[rand,:,0], color = 'C0')
        plt.subplot(312)
        plt.plot(valid_cut_times, cut_valid_waves_t[rand,:,1], color = 'C1')
        plt.subplot(313)
        plt.plot(valid_cut_times, cut_valid_waves_t[rand,:,2], color = 'C2')
        plt.subplots_adjust(hspace = 0)
        plt.show();

    ## Shift ##
    shift_len = cut_len - max_shift
    if debug: print('Shift len: ' + str(shift_len))
    time_offset = np.random.uniform(low = 0, high = max_shift, size = valid_size)
    shift_valid_waves_t = np.zeros((valid_size, int(shift_len * sr), 3)) 

    for ii, offset in enumerate(time_offset):
        bin_offset = int(offset * sr)
        start_bin = bin_offset 
        end_bin = int(start_bin + shift_len * sr)
        shift_valid_waves_t[ii, :, 0] = cut_valid_waves_t[ii, start_bin:end_bin, 0] 
        shift_valid_waves_t[ii, :, 1] = cut_valid_waves_t[ii, start_bin:end_bin, 1]
        shift_valid_waves_t[ii, :, 2] = cut_valid_waves_t[ii, start_bin:end_bin, 2]

    if debug: print('Shift waves t shape: ' + str(shift_valid_waves_t.shape))
    if debug:
        valid_shift_times = np.arange(0, shift_len, 1/sr)
        print('Rand: ' + str(rand))
        plt.figure(facecolor = 'white')
        plt.suptitle('Shifted validation data')
        plt.subplot(311)
        plt.plot(valid_shift_times, shift_valid_waves_t[rand,:,0], color = 'C0')
        plt.subplot(312)
        plt.plot(valid_shift_times, shift_valid_waves_t[rand,:,1], color = 'C1')
        plt.subplot(313)
        plt.plot(valid_shift_times, shift_valid_waves_t[rand,:,2], color = 'C2')
        plt.subplots_adjust(hspace = 0)
        plt.show();

    ### ----- Initialize the model and training setup ----- ###
    
    inp1 = tf.keras.layers.Input(shape = ((cut_len - max_shift)*sr, n_channels), name = 'input_layer') 
    e = tf.keras.layers.Conv1D(filters[1], 3, padding = 'same')(inp1) 
    e = tf.keras.layers.Dropout(drop_rate)(e, training = True)
    e = tf.keras.layers.MaxPooling1D(4, padding = 'same')(e)
    e = tf.keras.layers.Conv1D(filters[0], 3, padding = 'same')(e) 
    e = tf.keras.layers.Dropout(drop_rate)(e, training = True)
    e = tf.keras.layers.MaxPooling1D(4, padding = 'same')(e)
    e = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(100, return_sequences = False, dropout = 0.0, recurrent_dropout = 0.0))(e)
    #e = tf.keras.layers.Dense(2)(e)
    e = tf.keras.layers.Dense(1)(e)
    o = tf.keras.layers.Activation('linear', name = 'output_layer')(e)
    model = tf.keras.models.Model(inputs = [inp1], outputs = o)
    #model.summary()

    #model.compile(optimizer = 'Adam', loss = customLoss)
    model.compile(optimizer = 'Adam', loss = tf.keras.losses.MeanSquaredError())
    
    model_name = str(dataset) + '_' + str(nsamp) + 'samps_' + str(shift_len) + 's'
#     lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(factor = np.sqrt(0.1), cooldown = 0, patience = 4, min_lr = 0.5e-6)
    m_name = str(model_name) + '_{epoch:03d}.h5' 
#     filepath = os.path.join(save_dir, m_name)
#     early_stopping_monitor = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5)
#     checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath = filepath, monitor = 'val_loss', mode = 'auto', verbose = 1, save_best_only = True)
#     psv = PrintSomeValues()
#     callbacks = [lr_reducer, early_stopping_monitor, checkpoint, psv]
#     training_generator = dataGenerator(train_waves_t, train_mags, n_train_samp, window_len, cut_len, max_shift, sr, batch_size, n_channels)

#     ### ----- Train ----- ###

#     history = model.fit(training_generator, epochs = epochs_number, validation_data = (shift_valid_waves_t, valid_mags), callbacks = callbacks);

#     ### ----- Plot training curves ----- ###

#     plt.figure(facecolor = 'white')
#     plt.plot(history.history['loss'],label='Training Loss')
#     plt.plot(history.history['val_loss'], label='Validation Loss')
#     plt.legend()
#     plt.xlabel('Epoch')
#     plt.ylabel('Loss')
#     #plt.show()
#     plt.savefig(save_dir + '/loss_curves_' + m_name + '.png')
#     plt.close();
    
    ### ----- Load trained model ----- ###
    
    model_path = model_folder_path + training_dataset + '_' + str(nsamp) + 'samps_' + str(cut_len) + 's_window/'
#     print(model_path)
    list_of_models = glob.glob(model_path + '*.h5') # * means all if need specific format then *.csv
#     print(list_of_models)
    latest_model = max(list_of_models, key = os.path.getctime)
    print('Model: ' + str(latest_model))
    print('Loading weights...')
    
    model.load_weights(latest_model)

    ### ----- Make the predictions ----- ###

    #kdp = KerasDropoutPrediction(model)
    #predict, al_unc, ep_unc, comb = kdp.predict(shift_valid_waves_t, monte_carlo_sampling)
    print('Predicting...')
    predict = model.predict(shift_valid_waves_t)

    ### ----- Quick plot of the predictions vs. true magnitudes ----- ###

    fig4, ax = plt.subplots(facecolor = 'white')
    ax.scatter(valid_mags, predict, alpha = 0.4, facecolors = 'r', edgecolors = 'r')
    ax.plot([valid_mags.min(), valid_mags.max()], [valid_mags.min(), valid_mags.max()], 'k--', alpha=1, lw=2)
    ax.set_xlabel('Measured magnitude')
    ax.set_ylabel('Predicted magnitude')
#     plt.show()
    fig4.savefig(save_dir + '/scatter_' + m_name + '.png')
    plt.close();

    ### ----- Rename things ----- ###

    measured_mags = valid_mags
    predicted_mags = predict.flatten()

    ### ----- Calculate the error and standard deviation ----- ###

    errors = []

    for idx in range(len(predicted_mags)):
        predicted = predicted_mags[idx]
        measured = measured_mags[idx]
        error = predicted - measured
        errors.append(error)

    mean_error = np.mean(np.array(errors))
    std_error = np.std(np.array(errors))

    print('Mean error: ' + str(round(mean_error,3)))
    print('Error standard deviation: ' + str(round(std_error,2)))

    mean_errors.append(mean_error)
    std_errors.append(std_error)

    ### ----- Make the box and whisker plots with STF magnitude line ----- ###

    Tt = shift_len / 2
    M0_dyncm = Tt**3 * (0.625 * 10**23)
    Mw = ((2/3) * np.log10(M0_dyncm)) - 10.73 # M0 in dyne-cm

    print('Rupture duration: ' + str(Tt) + ' seconds')
    print('M0: ' + str(M0_dyncm) + ' dyne-cm')
    print('Mw: ' + str(round(Mw,2)))

    bins = np.arange(11,85,1)/10
    data_bins = []

    for abin in bins:
        i = np.where(valid_mags == abin)[0]
        predict_bin = np.array(predicted_mags[i])
        data_bins.append(predict_bin)

    fig = plt.figure(figsize =(14, 9), dpi = 300, facecolor = 'white')

    fig.suptitle('MLAAPDE ' + str(dataset) + ' agumented dataset, tested with ' + str(int(n_valid_samp)) + ' ' + str(shift_len) + 's window samples shifted up to 3s', fontsize = 18, y = 0.96, color = 'black')
    ax = fig.add_subplot(111)
    ax.set_facecolor('white')
    ax.text(x = 30, y = 8.8, s = 'Model: ' + m_name, fontsize = 13, color = 'black')
    ax.grid(which = 'major', axis = 'y')
    ax.grid(which = 'major', axis = 'x', markevery = [10,20,30,40,50])
    ax.set_ylim(1,8.6)

    bp = ax.boxplot(data_bins, notch = False, patch_artist = True)
    ax.axvline((Mw-1)*10, color = 'green', linestyle = '--', linewidth = 2) # Position = (magnitude - 1)*10

    for patch in bp['boxes']:
        patch.set_facecolor('lightblue')
        patch.set_edgecolor('blue')
    for median in bp['medians']:
        median.set(color ='blue', linewidth = 3)
    for whisker in bp['whiskers']:
        whisker.set(color ='blue', linewidth = 1)
    for cap in bp['caps']:
        cap.set(color ='blue', linewidth = 1)
    for flier in bp['fliers']:
        flier.set(marker ='+', color ='blue', alpha = 0.5)

    bins_list = bins.tolist()
    ax.set_xticklabels(bins_list, fontsize = 14, color = 'black')
    ax.set_yticklabels([1, 2, 3, 4, 5, 6, 7, 8], fontsize = 14, color = 'black')
    ax.set_ylabel('Predicted magnitude', fontsize = 16, color = 'black')
    ax.set_xlabel('Measured magnitude', fontsize = 16, color = 'black')
    ax.xaxis.set_major_locator(plt.MaxNLocator(8))
    ax.plot((1.1,70),(1.1,8),'r--', linewidth = 3, alpha = 0.5)
    ax.text(s = 'Testing results', x = 2, y = 8, fontsize = 18, backgroundcolor = 'lightskyblue', color = 'black')
    ax.text(s = 'STF magnitude: ' + str(round(Mw,2)), x = 2, y = 7.5, fontsize = 18, backgroundcolor = 'lightgreen', color = 'black')

#     plt.show()
    plt.savefig(save_dir + '/boxplot_durline_' + m_name + '.png', format = 'PNG', facecolor = 'white', transparent = False)
    plt.close();

Cut len: 7
Model: /home/sdybing/mlaapde/training/v1b_2920609samps_7s_window/v1b_2920609samps_7s_033.h5
Loading weights...
Predicting...
Mean error: -2.041
Error standard deviation: 1.05
Rupture duration: 0.5 seconds
M0: 7.8125e+21 dyne-cm
Mw: 3.87


<ipython-input-25-ea23fbc32ef3>:231: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels([1, 2, 3, 4, 5, 6, 7, 8], fontsize = 14, color = 'black')


Cut len: 8
Model: /home/sdybing/mlaapde/training/v1b_2920609samps_8s_window/v1b_2920609samps_8s_010.h5
Loading weights...
Predicting...
Mean error: -0.93
Error standard deviation: 1.06
Rupture duration: 1.0 seconds
M0: 6.25e+22 dyne-cm
Mw: 4.47


<ipython-input-25-ea23fbc32ef3>:231: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels([1, 2, 3, 4, 5, 6, 7, 8], fontsize = 14, color = 'black')


Cut len: 9
Model: /home/sdybing/mlaapde/training/v1b_2920609samps_9s_window/v1b_2920609samps_9s_011.h5
Loading weights...
Predicting...
Mean error: 0.165
Error standard deviation: 1.02
Rupture duration: 1.5 seconds
M0: 2.109375e+23 dyne-cm
Mw: 4.82


<ipython-input-25-ea23fbc32ef3>:231: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels([1, 2, 3, 4, 5, 6, 7, 8], fontsize = 14, color = 'black')


Cut len: 10
Model: /home/sdybing/mlaapde/training/v1b_2920609samps_10s_window/v1b_2920609samps_10s_025.h5
Loading weights...
Predicting...
Mean error: 0.3
Error standard deviation: 0.96
Rupture duration: 2.0 seconds
M0: 5e+23 dyne-cm
Mw: 5.07


<ipython-input-25-ea23fbc32ef3>:231: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels([1, 2, 3, 4, 5, 6, 7, 8], fontsize = 14, color = 'black')


Cut len: 11
Model: /home/sdybing/mlaapde/training/v1b_2920609samps_11s_window/v1b_2920609samps_11s_024.h5
Loading weights...
Predicting...


In [ ]:
np.savetxt(models_figs_path + data + '/fulldataset_validsamp_meanerrors.txt', np.array(mean_errors))
np.savetxt(models_figs_path + data + '/fulldataset_validsamp_stderrors.txt', np.array(std_errors))

In [ ]:
### ----- Plot error and std for all windows ----- ###

shift_lengths = []

for cut_len in cut_lens:
    shift_len = cut_len - max_shift
    shift_lengths.append(shift_len)
    
plt.figure(figsize = (10, 6), facecolor = 'white')
plt.title('Testing errors/stds: models trained with\n100,000 augmented samples shifted up to 3 seconds', fontsize = 16)
plt.errorbar(shift_lengths, mean_errors, yerr = std_errors, fmt = '.', markersize = 10, ecolor = 'C1', capsize = 3, label = 'Error bars show 1 standard\ndeviation above each point and\n1 standard deviation below')
plt.scatter(shift_lengths, mean_errors, color = 'C0')
plt.grid()
plt.xlabel('Window length (s)', fontsize = 14)
plt.ylabel('Mean error\n(predicted - measured magnitude)', fontsize = 14)
plt.xticks(fontsize = 13)
plt.yticks(fontsize = 13)
plt.legend(fontsize = 12)
plt.axhline(0, color = 'black', linestyle = '--', alpha = 0.75)

#plt.show()
plt.savefig(models_figs_path + data + '/fulldataset_validsamp_all_errors_stds.png', format = 'PNG', facecolor = 'white', transparent = False)
plt.close();